## Q1.

Add methods `__iter__` to your project Time Series class to iterate over values, a method `itertimes` to iterate over times, a method `itervalues` to iterate over values, and a method `iteritems` to iterate over time-value pairs. (This is a similar interface to python dictionaries). To test these, check both the types of the results and the answers you expect.

In [1]:
#your code here


## Q2.

An online mean and standard deviation algorithm.

Below is a function to generate a potentially infinite stream of 1-D data.

In [5]:
from random import normalvariate, random
from itertools import count
def make_data(m, stop=None):
    for _ in count():
        if stop and _ > stop:
            break
        yield 1.0e09 + normalvariate(0, m*random() )
        

Here is an implementation of an online mean algorithm..see http://www.johndcook.com/blog/standard_deviation/ and the link to http://www.johndcook.com/blog/2008/09/26/comparing-three-methods-of-computing-standard-deviation/ in-between. (Convince yourselves of the formulas...)

In [6]:
def online_mean(iterator):
    n = 0
    mu = 0
    for value in iterator:
        n += 1
        delta = value - mu
        mu = mu + delta/n
        yield mu

We use out generator functions to implement iterators:

In [15]:
g = list(make_data(5, 10))
print(g)

[1000000000.1231574, 999999999.6919035, 999999999.0117985, 1000000000.0126944, 1000000005.5084858, 1000000000.0169846, 999999994.628876, 999999997.8346056, 1000000000.388472, 1000000003.990709, 999999999.4109826]


In [16]:
ab = online_mean(g)
list(ab)

[1000000000.1231574,
 999999999.9075304,
 999999999.6089531,
 999999999.7098885,
 1000000000.8696079,
 1000000000.727504,
 999999999.8562714,
 999999999.6035632,
 999999999.6907753,
 1000000000.1207687,
 1000000000.0562427]

In [8]:
g = online_mean(make_data(5, 100))
print(type(g))
list(g)

<class 'generator'>


[1000000009.0726615,
 1000000004.6368046,
 1000000003.5791572,
 1000000002.1090913,
 1000000001.3684376,
 1000000001.9162925,
 1000000001.654492,
 1000000001.4053586,
 1000000001.6552278,
 1000000001.4617195,
 1000000001.359219,
 1000000001.1902303,
 1000000001.4465213,
 1000000001.3499578,
 1000000001.6412671,
 1000000001.5531092,
 1000000001.5814182,
 1000000001.2408297,
 1000000001.0314033,
 1000000000.5478767,
 1000000000.597401,
 1000000000.6441196,
 1000000000.6398298,
 1000000000.6003413,
 1000000000.8441517,
 1000000000.7470005,
 1000000000.6819477,
 1000000000.6536723,
 1000000000.5492172,
 1000000000.545035,
 1000000000.3491234,
 1000000000.353828,
 1000000000.6674933,
 1000000000.6406887,
 1000000000.6140962,
 1000000000.5436988,
 1000000000.5762368,
 1000000000.5484923,
 1000000000.4401009,
 1000000000.422202,
 1000000000.4205872,
 1000000000.3431703,
 1000000000.0627176,
 1000000000.0739279,
 1000000000.0799198,
 1000000000.0943652,
 999999999.995532,
 999999999.9536223,
 

### 2.1

Implement the standard deviation algorithm as a generator function as

```python
def online_mean_dev(iterator):
    BLA BLA
    if n > 1:
        stddev = math.sqrt(dev_accum/(n-1))
        yield (n, value, mu, stddev)
```

In [40]:
import math
def online_mean_dev(iterator):
    # // initialize
    n = 1
    dev_accum = 0
    
    # // begin mean calculation
    mean_generator = online_mean(iter(iterator))
    mu_prev = next(mean_generator)
    
    # // compute standard deviation online
    for value in iterator:
        n += 1
        mu = next(mean_generator)
        dev_accum += (value - mu_prev)*(value - mu)
        mu_prev = mu
        stddev = math.sqrt(dev_accum/(n-1))
        yield (n, value, mu, stddev)

Here we make 100000 element data, and run this iterator on it (imagine running this on a time-series being slowly read from disk

In [41]:
data_with_stats = online_mean_dev(make_data(5, 100000))

## Q3.

Let's do Anomaly detection. Write a routine `is_ok`:

```python
def is_ok(level, t)
```

which takes a tuple like the one yielded by your code above and returns True if the value is inbetween `level`-$\sigma$ of the mean.

In [45]:
#your code here
def is_ok(level,t):
    return (t[1] > t[2] - (level * t[3])) and (t[1] < t[2] + (level * t[3]))

We use this function to create a predicate passed through to `itertools.filterfalse` which is then used to obtain an iterator on the anomalies.

In [46]:
from itertools import filterfalse
pred = lambda t: is_ok(5, t)
anomalies = filterfalse(pred, data_with_stats)

We materialize the anomalies...

In [47]:
list(anomalies)#materialize

[(7581, 1000000016.471212, 999999999.9617169, 2.918464847772603),
 (9019, 999999985.0258793, 999999999.9874806, 2.9193850510182866),
 (11561, 1000000015.5639933, 999999999.9708933, 2.910737715492603),
 (16754, 999999984.859731, 999999999.982145, 2.91946151042329),
 (18965, 999999985.3692993, 999999999.985391, 2.9177476069036548),
 (20523, 1000000014.762635, 999999999.9950593, 2.9208613250689766),
 (20560, 1000000015.5623449, 999999999.9949514, 2.9229980742282007),
 (20657, 999999983.4862732, 999999999.9977247, 2.9250648211430645),
 (27846, 999999985.3283511, 999999999.9971168, 2.8939398198417146),
 (28351, 1000000018.453933, 999999999.9962655, 2.8929166924350334),
 (38545, 1000000014.5901663, 999999999.9971038, 2.887784301746257),
 (41267, 999999983.1233371, 1000000000.0002006, 2.8854474102282084),
 (42980, 999999984.7976887, 999999999.9962395, 2.8860507643294917),
 (46793, 1000000015.1378298, 999999999.9944646, 2.8830915845726843)]

## To think of, but not hand in

What kinds of anomalies will this algorithm pick up? What kinds would a shorter "window" of anomaly detection, like 100 points around the time in question pick? How might you create an algorithm which does window based averaging? (hint: the window size is small compared to the time series size). 

Finally think a bit of how you might implement all of this in a production environment..remember that data streaming in might get backed up when you handle an anomaly.

(Some inspiration might accrue if you look at the docs for `collections.deque`).

**Answer:**
This algorithm will pick up anomalies that are very far from the mean, which occur naturally over much sampling.

A shorter window of anomaly detection would detect more anomalies, as the variance would be more volatile. The window based algorithm would hold values over the course of the window, and subtract them out in the same way that they were added in.